# Lagged Features

In [1]:
# TODO: I read in Franzi's merged/final DF as df_full. This name must be replaced by her name in the following cells

In [5]:
import pandas as pd
df_full = pd.read_parquet("data_s2000_currently.parquet")
df_full = df_full[df_full["shopper"]<3]

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect,...,week_basket_size,week_basket_value,mean_basket_size,mean_basket_value,avg_offered_dis_week,avg_used_dis_week,customer_mean_product_price,customer_discount_buy_share,product_dis_sells_share,customer_product_sales_share
0,0,0,5,504.70,30.0,1,0,0,1,0,...,10.0,5908.0,8.992208,5273.981818,6.800000,0.0,585.901427,0.998703,1.0,-1.000000
1,0,0,16,743.00,0.0,0,0,0,0,0,...,10.0,5908.0,8.992208,5273.981818,6.800000,0.0,585.901427,0.998703,1.0,0.005188
2,0,0,27,407.40,30.0,1,0,0,1,0,...,10.0,5908.0,8.992208,5273.981818,6.800000,0.0,585.901427,0.998703,1.0,-1.000000
3,0,0,35,391.30,35.0,1,0,0,1,0,...,10.0,5908.0,8.992208,5273.981818,6.800000,0.0,585.901427,0.998703,1.0,-1.000000
4,0,0,52,607.00,0.0,0,0,0,0,0,...,10.0,5908.0,8.992208,5273.981818,6.800000,0.0,585.901427,0.998703,1.0,0.011673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541010,89,2,179,444.75,25.0,1,0,0,1,0,...,3.0,1910.0,6.554702,3735.122841,7.272727,0.0,571.195402,0.998084,1.0,-1.000000
3541011,89,2,217,657.00,0.0,0,0,0,0,0,...,3.0,1910.0,6.554702,3735.122841,7.272727,0.0,571.195402,0.998084,1.0,0.022989
3541012,89,2,218,628.15,15.0,1,0,0,1,0,...,3.0,1910.0,6.554702,3735.122841,7.272727,0.0,571.195402,0.998084,1.0,0.001916
3541013,89,2,228,531.00,0.0,0,0,0,0,0,...,3.0,1910.0,6.554702,3735.122841,7.272727,0.0,571.195402,0.998084,1.0,0.042146


In [44]:
# from module_lags import LagCalculator
import module_lags

In [32]:
purchase_lags = module_lags.LagCalculator(
    df_full[["shopper", "product", "week", "product_bought"]]
    .sort_values(by=["shopper", "product", "week"], ascending=[True, True, True])
    .reset_index(drop=True)
)

In [51]:
lags = purchase_lags.calculate_lags()
lags.iloc[3001:3006]

0% done.
10% done.
20% done.
30% done.
40% done.
50% done.
60% done.
70% done.
80% done.
90% done.
100% done.


,shopper,product,week,product_bought,lag_weeks_of_product_per_customer
3001,1,156,3,0,-1
3002,1,156,8,1,-1
3003,1,156,9,1,1
3004,1,156,13,1,4
3005,1,156,18,0,5


In [41]:
purchase_temporal_distribution = purchase_lags.calculate_purchase_temporal_distribution(lags)
purchase_temporal_distribution.head()

,shopper,product,purchase_temporal_distribution
0,0,4,50.250000
1,0,11,59.666667
2,0,15,10.000000
3,0,16,45.000000
4,0,21,86.000000


In [42]:
avg_no_weeks_between_two_purchases = purchase_lags.calculate_avg_no_weeks_between_two_purchases(lags)
avg_no_weeks_between_two_purchases.head()

,shopper,product,avg_no_weeks_between_two_purchases
0,0,4,5.625000
1,0,11,6.777778
2,0,15,-1.000000
3,0,16,15.500000
4,0,21,-1.000000


In [ ]:
# export
lags.to_parquet()
purchase_temporal_distribution.to_parquet()
avg_no_weeks_between_two_purchases.to_parquet()

In [20]:
# example
lags[(lags["shopper"]==333) & (lags["week"]==66)]

,shopper,product,week,product_bought,lag_weeks_of_product_per_customer
2632783,333,20,66,0,-1
2632784,333,41,66,0,5
2632785,333,48,66,0,1
2632786,333,51,66,0,-1
2632787,333,60,66,1,3
2632788,333,80,66,0,-1
2632789,333,129,66,0,8
2632790,333,132,66,0,6
2632791,333,137,66,1,3
2632792,333,144,66,1,12
